# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Install Packages
!pip install hvplot geoviews datashader cartopy pyproj

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [3]:
# geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alice springs,-23.7000,133.8833,35.75,13,91,7.20,AU,1696235273
1,1,zavoronezhskoye,52.8801,40.5526,12.88,88,100,3.13,RU,1696235520
2,2,saurimo,-9.6608,20.3916,32.21,36,88,0.99,AO,1696235520
3,3,portland,45.5234,-122.6762,13.12,84,40,0.45,US,1696234955
4,4,minas de marcona,-15.2119,-75.1103,13.37,92,88,4.09,PE,1696235520


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Organize point size into humidity and convert astype
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
humidity_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 700,
    size = "Humidity",
    color="City",
    alpha=0.70
).opts(title="Humidity Geoview Map")


# Display the map
humidity_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,hermanus,-34.4187,19.2345,21.84,48,0,1.76,ZA,1696235521
47,47,saldanha,-33.0117,17.9442,24.03,36,0,0.32,ZA,1696235525
68,68,hawaiian paradise park,19.5933,-154.9731,22.80,95,0,1.54,US,1696235528
138,138,valley city,46.9233,-98.0032,21.29,64,0,4.12,US,1696235536
142,142,el haouaria,37.0500,11.0144,25.01,59,0,4.00,TN,1696235536


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\SauceyClaps\AppData\Local\Temp\ipykernel_8284\3768208391.py:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\SauceyClaps\AppData\Local\Temp\ipykernel_8284\3768208391.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
17,hermanus,ZA,-34.4187,19.2345,48,
47,saldanha,ZA,-33.0117,17.9442,36,
68,hawaiian paradise park,US,19.5933,-154.9731,95,
138,valley city,US,46.9233,-98.0032,64,
142,el haouaria,TN,37.0500,11.0144,59,
144,bredasdorp,ZA,-34.5322,20.0403,38,
145,redwood falls,US,44.5394,-95.1169,68,
164,plettenberg bay,ZA,-34.0527,23.3716,42,
170,alpnach,CH,46.9402,8.2743,72,
217,misratah,LY,32.3754,15.0925,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: Aloe guest house
saldanha - nearest hotel: Hoedjesbaai
hawaiian paradise park - nearest hotel: No hotel found
valley city - nearest hotel: AmericInn
el haouaria - nearest hotel: Villa Zembra Maison d'hôtes
bredasdorp - nearest hotel: Victoria Hotel
redwood falls - nearest hotel: Travelodge by Wyndham Redwood Falls
plettenberg bay - nearest hotel: Milkwood Manor
alpnach - nearest hotel: Schlüssel
misratah - nearest hotel: الفندق السياحى
berriozar - nearest hotel: Hotel Pamplona Plaza
bechar - nearest hotel: Hôtel Madania
olevano sul tusciano - nearest hotel: Hotel Cristal
tamanrasset - nearest hotel: فندق الأمان
colonia - nearest hotel: Wasserturm Hotel Cologne
queenstown - nearest hotel: Queens Hotel
bojnurd - nearest hotel: مرکز آموزشی رفاهی فرهنگیان بجنورد
beruniy - nearest hotel: Miymanxana
kroonstad - nearest hotel: No hotel found
hami - nearest hotel: Хами
berrouaghia - nearest hotel: Andalous
hilo - nearest hotel: Dolphin Bay Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
17,hermanus,ZA,-34.4187,19.2345,48,Aloe guest house
47,saldanha,ZA,-33.0117,17.9442,36,Hoedjesbaai
68,hawaiian paradise park,US,19.5933,-154.9731,95,No hotel found
138,valley city,US,46.9233,-98.0032,64,AmericInn
142,el haouaria,TN,37.0500,11.0144,59,Villa Zembra Maison d'hôtes
144,bredasdorp,ZA,-34.5322,20.0403,38,Victoria Hotel
145,redwood falls,US,44.5394,-95.1169,68,Travelodge by Wyndham Redwood Falls
164,plettenberg bay,ZA,-34.0527,23.3716,42,Milkwood Manor
170,alpnach,CH,46.9402,8.2743,72,Schlüssel
217,misratah,LY,32.3754,15.0925,57,الفندق السياحى


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotels_found = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    color="City",
    alpha=.70,
    hover_cols=["Hotel Name", "Country"]
).opts(title="Hotels within 10,000 meters of city")

# Display the map
hotels_found

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)